In [1]:
!pip install -q sentence-transformers
!pip install -q wikipedia-api
!pip install -q numpy
!pip install -q scipy

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

In [3]:
from wikipediaapi import Wikipedia
wiki = Wikipedia('RAGBot/0.0', 'en')
doc = wiki.page('Silk_Sonic').text
paragraphs = doc.split('\n\n') # chunking

In [4]:
import textwrap

In [5]:
for i, p in enumerate(paragraphs):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")

-----------------------------------------------------------------
Silk Sonic is an American musical superduo composed of musicians Bruno Mars and Anderson .Paak. The
duo released their debut single, "Leave the Door Open", in March 2021.  They later released "Skate"
in July 2021, followed by third single, "Smokin out the Window", and their debut album, An Evening
with Silk Sonic, in November 2021. The album peaked at number two on the Billboard 200, while the
first three singles reached the top twenty on the Billboard Hot 100, with "Leave the Door Open"
spending 18 weeks in the top ten and two non-consecutive weeks at the top spot. The song later won
four Grammy Awards, including Record of the Year and Song of the Year.
-----------------------------------------------------------------
-----------------------------------------------------------------
History 2017–present: Beginnings, breakthrough, and An Evening with Silk Sonic In 2017, Anderson
.Paak was the opening act for Bruno Mars o

In [6]:
docs_embed = model.encode(paragraphs, normalize_embeddings=True)

In [7]:
docs_embed.shape

(7, 768)

In [8]:
docs_embed[0]

array([ 3.75271812e-02, -4.85358089e-02,  1.69990137e-02, -3.89322564e-02,
        3.23089138e-02, -2.87614632e-02,  5.33560961e-02, -2.64595798e-03,
        6.46743551e-03, -2.70928722e-02,  1.21302474e-02, -6.60750270e-02,
        2.52066273e-02, -8.09831917e-03,  7.34939799e-03, -7.40109235e-02,
       -3.19186226e-02, -7.15053082e-02, -9.12133325e-03, -3.15642096e-02,
       -3.63807864e-02, -2.62687970e-02,  2.01912187e-02,  2.15709768e-02,
        7.87328836e-03, -1.64775103e-02,  3.33012976e-02,  3.91034782e-02,
        2.08830256e-02,  2.37077381e-02,  3.84917632e-02, -2.26394180e-02,
       -1.33538246e-02,  8.78478494e-03, -3.84362824e-02,  1.81481913e-02,
       -2.55182721e-02,  7.24443868e-02, -3.06680296e-02,  3.12690772e-02,
       -3.91105600e-02,  4.75264736e-04, -2.31090542e-02,  6.34418381e-03,
       -7.38116130e-02, -1.07810153e-02, -1.67395361e-02, -4.34494652e-02,
        1.76723041e-02, -2.03240523e-03,  6.76081935e-03,  1.85174905e-02,
       -2.38388367e-02,  

In [25]:
query = "What is the music style of taylor swift"
query_embed = model.encode(query, normalize_embeddings=True)

In [26]:
query_embed.shape

(768,)

In [27]:
import numpy as np
similarities = np.dot(docs_embed, query_embed.T)

In [28]:
similarities.shape

(7,)

In [29]:
similarities

array([0.39204654, 0.3076191 , 0.45517075, 0.24736863, 0.28750896,
       0.22191258, 0.23579693], dtype=float32)

In [30]:
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()

In [31]:
top_3_idx

[2, 0, 1]

In [32]:
most_similar_documents = [paragraphs[idx] for idx in top_3_idx]

In [33]:
CONTEXT = ""
for i, p in enumerate(most_similar_documents):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")
  CONTEXT += wrapped_text + "\n\n"                   

-----------------------------------------------------------------
Musical style Silk Sonic's musical style has been described as R&B, soul, funk, pop, hip hop, and
smooth soul. Its sound has also been described as disco-inspired. Of An Evening with Silk Sonic's
sound, The Ringer wrote, "though styled as a 1960s to '70s retro endeavor, Silk Sonic isn't a work
of nostalgia but rather a fusion: funk, rap, and R&B as they've sounded in heavy rotation as
recently as the 2000s." Sputnikmusic called the album "rich and authentic, existing at the
irresistible intersection between 70s funk nostalgia and the luxuries of a modern day studio. As a
result, An Evening With Silk Sonic lives up to its billing as a true experience: it’s sexy, ever-so-
smooth, and radiates confidence and charisma."
-----------------------------------------------------------------
-----------------------------------------------------------------
Silk Sonic is an American musical superduo composed of musicians Bruno Mars 

In [34]:
query = "What is the music style of taylor swift?"

In [35]:
prompt = f"""
use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

In [2]:
!pip install -q openai
!pip install python-dotenv

In [3]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()

True

In [4]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [39]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": prompt},
  ]
)

In [40]:
print(response.choices[0].message.content)

I don't know. The provided context focuses on the musical style of Silk Sonic and does not provide information about Taylor Swift's music style.
